# Homework 3 
# Plotting Politics By Proxy
<b>ASSIGNED: Friday, 29 Mar 2019.</b>   
<b>DUE: Wednesday, 9 April 2019 at 5 pm</b>

All students on both tracks must complete this homework. 

## Goal:
Using unsupervised learning we're going to produce a plot of US politicians via their individual language usage on Twitter, starting with ingesting the tweets, preparing these tweets for PCA, and finally performing PCA. We're going to use this exercise to reflect, analyze, and explore how such practices both reify and challenge our contemporary understanding of community and the public discourse. 

More specifically, you're going to
1. tokenize and get word counts for a set of tweets;
2. produce a PCA plot of twitter user accounts based on word usage;
3. explore how word usage is (or isn't) a good proxy for political affliation for people in public office.

#### To do this homework, you will make use of the following files:
1. 10K2016tweets.csv
2. pol_aff.csv

## Instructions 
This assignment is to be done <b>on your own</b>, but you can talk about the assignment with your classmates if you get stuck. (Be sure to list the students you spoke with about this assignment in the space provided below.) Feel free to also use [stackoverflow](https://stackoverflow.com/) but please provide citation and link to the specific answer if you do this. You may also visit Will Yumou during his TA office hours or email him with questions.

Provide your code to justify your answer to each question. Your code must run with the "10K2016tweets.csv" and "pol_aff.csv" files as originally provided to you. 

Be sure to rename this homework notebook so that it includes your name. 

### List any students you talked with about this assignment here:
1. [person 1]
2. [person 2]
3. etc.


# Homework Problems


## Step 1: Getting Word Frequencies For All Words in each tweet

### question 1 [5 points]
1. Ingest the file 10K2016tweets.csv into pandas as a dataframe entitled `tweets`. Note that each tweet is a row. We've provided the libraries you'll need for this assignment.
2. Use code provdied to drop rows with NAN values.
3. Use `tweets.tail()` to get a sense of what the data frame is. 

We've given you some libraries you'll need for this assignment.

In [1]:
import pandas as pd
from sklearn.decomposition import PCA  
import matplotlib.pyplot as plt
import matplotlib 
import numpy as np

In [ ]:
# insert your code to read tweet csv into a dataframe called "tweets". 



In [ ]:
# Now run this code to drop rows with NAN values
tweets.dropna(inplace=True)
tweets = tweets.reset_index(drop=True)

In [ ]:
# Now use tail() to see last five rows of dataframe "tweets"


### question 2 [10 points]
1. What day and time was the oldest tweet in the data set posted? Hint: use `tweets['created_at'].min()`. Also, be sure to make use of the `to_datetime()` command. 
2. What day and time was the most recent tweet in the data set posted?
3. How many *tweets* does the data set have? 
4. How many unique twitter users does the data set have? Hint: use the `nunique()` command. 

In [ ]:
# insert your code here.


### question 3 [30 points]
Use *sklearn's CountVectorizer* to produce a table of word counts in which each tweet is a row and each word is a column <b>for the first 100 tweets in your dataset</b>. Name this table "wc".

#### For an EXAMPLE of using sklearn countvectorizer to produce this `wc`, see the code in the following 5 cells. Note that this example just uses three made-up tweets so that it's easier to see what's happening.  

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# Here we list a few sample strings, where each 
# string can be considered one text. For the tweets 
# dataframe you'll to produce a list in which each tweet is one string.  
# The string "The animal is large" is tweet 0 below; the string
# "is is is" is tweet 1 below; etc.
three_tweets = ["The animal is large.", "is is is", "The can't"]

# create vectorizer 
cv = CountVectorizer()

# tokenize texts & get vocabulary
cv.fit(three_tweets) #note that the sklearn tokenizer turns "can't" into "can"

# create a dictionary where each word is given an index number.
# If you need to know what word a number represents, refer back to this dictionary
word_index = cv.vocabulary_
word_index

{'the': 4, 'animal': 0, 'is': 2, 'large': 3, 'can': 1}

In [3]:
# to find out what column number a particular word is, say, "animal" we can use 
word_index["animal"]

0

In [4]:
# generate a table of tweets vs words
word_counts = cv.transform(three_tweets)

# and get the "shape" of this table like this:
word_counts.shape
# where the 3 rows are the three tweets and the 
# 5 columns are the 5 words in this corpus 

(3, 5)

In [5]:
# to get dataframe of with words instead of word index numbers:
feature_names = cv.get_feature_names()
word_counts_df = pd.SparseDataFrame(word_counts, columns=feature_names).fillna(0)
word_counts_df

,animal,can,is,large,the
0,1.0,0.0,1.0,1.0,1.0
1,0.0,0.0,3.0,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0


In [6]:
# Finally, you can dump the word_counts table into an array
# where each row is a tweet, each column is a word type, and each
# element is a word token count (where each word is denoted by a number
# as defined in the word_index above). For the difference between word types and word tokens, 
# see "type-token distinction" in wikipedia.
wc = word_counts.toarray()
wc

array([[1, 0, 1, 1, 1],
       [0, 0, 3, 0, 0],
       [0, 1, 0, 0, 1]])

<b>This concludes the example code.</b> 


Now do the same thing we did for the example above, but substitute the "three_texts" list in the above example with the first 100 tweets that appear in your data set.

You can get a list called "all_tweets" of all your tweet texts using the following code: `all_tweets = tweets['tweet_text']`. To get the first 100 tweets, you can use a python slice: `all_tweets[0:100]`. We've provided this code to get you started.

<b>At the very end of your code for this question, you should output `wc` just like we did in the example above.</b> 

#### Put your code for question 3 below:

In [ ]:
# extract all_tweets from df as a list of strings. 
all_tweets = tweets['tweet_text']
hundred_tweets = all_tweets[0:100]


## Step 2: Find word usage for each twitter account

### question 4 [30 points]
The table wc give us the word counts for all tweets, <b>but what we really want is a table of word counts for each twitter account in our first 100 tweets</b>.

To do this: 
<b>Make a new array called `wc_accounts` where each row is a twitter account user and the columns are word token counts for unique word types.</b> Note that the dataframe "tweets" has a `user_id` provided for each tweet. 

To make `wc_accounts` use a FOR loop which runs over all `user_id`s. For each loop in the FOR loop you should do the following:
1. get all the rows of one user's tweets from `tweets` in a list and convert this to a numpy array
2. put this array of tweet rows into the `wc` array you built in question 3 (which will return all the words of user tweets), and use the `sum()` command on this resultant array to produce a single row with total word counts for one user
3. take this single row of total word counts for one user and add this row `temp_results;

Once the for loop is completed, you can save `temp_results` as `wc_accounts`.

To get you started, here's a way to get a list of all `user_id`s, which you need to run your FOR loop: `twitter_users = tweets['user_id'].unique()`. We've provided this code.

We've also provided you with some additional code. Your job is to fill in the appropriate commands in the FOR loop provided.

Once you're done, be sure to output `wc_accounts`. (It will resemble the output for `wc`, but with different numbers.) We've provided this code for you too. 

#### Put your code for question 4 below: 

In [ ]:
#get twitter users from the dataframe
twitter_users = tweets['user_id'].unique() # this gets all twitter user IDs
temp_results = []                          # an empty list

for user in twitter_users:
    # insert your code for FOR loop here
    # ...
    # ...
    # ...
    # ...
    # ...
    
wc_accounts = np.array(temp_results)
wc_accounts

## Step 3: Perform PCA on wc_accounts

Now that we have wc_acounts, we'd like to plot this to see how all these accounts use language. But how plot this in two dimensions? Let's use a form of unsupervised learning called PCA, where each account (i.e., each row) represents one data point. This is the same as the PCA we performed on the Iris and Spearman data sets in our labs. We can plot our twitter account data in exactly in the same way we did then.   

However, we have one additional wrinkle in our task. It'd be helpful to know which political affliation each individual twitter account has. Fortunately, I have this data for you! In pol_aff.csv I have included user_ID and their respective political affliation. Use this csv to get the political association of each user account. Note that this file includes user_IDs that don't exist in your corpus of tweets so you'll need to selectively add political affliation based on user IDs. 

### question 5 [40 points]
Perform PCA on individual user accounts using the data in `wc_accounts`. Plot this in 2 dimensions where each data point represents one twitter account. Color code each data point according to account political affliation, with red = republicans and blue = democrats. To make overlapping data points easier to see, reduce the opacity (i.e., the "alpha" parameter) to 0.3. 

You can break this up into several steps:

<b>Step 1</b>: Generate a dataframe called `merged_data` with two columns: `user_id` (that comes from `tweets`) and `affliation` (that comes from `pol_aff.csv`). You should have the same number of rows as you have twitter users that you found in question 2. Here's the commands you'll need:
1. `read_csv()` (to read in `pol_aff.csv`)
2. `unique()` (to get unique twitter users in `tweets` & put them in a new data frame)
3. `merge()` (to merge dataframe with `user_id` with dataframe with `affliation`)
4. `drop()` (to drop any columns you have in your final dataframe that you don't need)


<b>Step 2</b>: 
<b>Next you want to produce a dataframe called `Graph` in which the rows are twitter users, and the three columns are the twitter user's respective principal component 1 coordinates, principal component 2 coordinates, and affliation.</b> To get the principal components, you'll need to do PCA on `wc_accounts` (from question #4) and then place the results of this PCA into a dataframe that you merge with `affliation` column in step one.

<b>Step 3</b>: Once you've constructed the `Graph` dataframe in Step 2, just run the code provided to generate your PCA plot. 



#### Put your code for question 5 below:

In [ ]:
# your code for step 1


In [ ]:
# your code for step 2


In [ ]:
# Code to run for Step #3 -- DON'T CHANGE THIS, JUST RUN IT 
# RUN TO PLOT PCA DATA ONCE YOU HAVE CONSTRUCTED THE GRAPH DATAFRAME

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['democrat', 'republican']
colors = ['b','r']
for target, color in zip(targets,colors):
    indicesToKeep = Graph['affliation'] == target
    ax.scatter(Graph.loc[indicesToKeep, 'principal component 1']
               , Graph.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha =.3)
ax.legend(targets)
ax.grid()